# 🌇 Welcome to the `visualiser` module!

In this notebook we'll explore how to finally visualise the result of an entire workflow (examples `[1-5]`) into a maps—static and interactive visualisers.

Let’s make an enriched layer pop!

**Data source used**:
- PLUTO data from NYC Open Data. https://www.nyc.gov/content/planning/pages/resources/datasets/mappluto-pluto-change

**What you’ll learn**:
- Setting up an enriched layer.
- Drawing a static map.
- Creating an interactive map with e.g a dark theme.

Let’s make some pretty maps! 🗺️

In [ ]:
import urban_mapper as um

# Kick off UrbanMapper
mapper = um.UrbanMapper()

## Preparing the Data and Layer

First, let’s load data, create a layer, and enrich it with average building floors.

Note that:

- Loader example can be seen in `examples/Basics/loader.ipynb`
- Urban Layer example can be seen in `examples/Basics/urban_layer.ipynb`
- Enricher example can be see in `examples/Basics/enricher.ipynb`
- Imputer example can be seen in `examples/Basics/imputer.ipynb`
- Filter example can be seen in `examples/Basics/filter.ipynb`

In [ ]:
######
#
# We will on purpose do inline chaining of the methods to fasten the process.
#
######

# Load data
data = mapper.loader.from_file("./pluto.csv").with_columns("longitude", "latitude").load()

# Create urban layer
layer = mapper.urban_layer.with_type("streets_intersections").from_place("Downtown Brooklyn, New York City, USA").build()

# Impute your data if they contain missing values
data = mapper.imputer.with_type("SimpleGeoImputer").on_columns("longitude", "latitude").transform(data, layer)

# Filter your data if they contain data beyond the bounding box your urban layer above
data = mapper.filter.with_type("BoundingBoxFilter").transform(data, layer)

# Map data to nearest layer
_, mapped_data = layer.map_nearest_layer(
    data,
    longitude_column="longitude",
    latitude_column="latitude",
    output_column="nearest_intersection",
)

# Enrich layer
enricher = (
    mapper.enricher
    .with_data(group_by="nearest_intersection", values_from="numfloors")
    .aggregate_by(method="mean", output_column="avg_floors")
    .build()
)

enriched_layer = enricher.enrich(mapped_data, layer)

## Static Visualisation

Now that we've cooked the various ingredients, let’s whip up a static map with Matplotlib to see our enriched layer.

In [ ]:
# Static visualisation
fig_static = (
    mapper
    .visual # From the visualiser module
    .with_type("Static")  # With type Static
    .show(columns=["avg_floors"])  # Show the avg_floors column
    .render(enriched_layer.get_layer()) # Render the enriched layer
)

## Interactive Visualisation

Now, let’s go fancy with an interactive Folium map—dark theme, because why not?

In [ ]:
# Interactive visualisation
fig_interactive = (
    mapper
    .visual # From the visualiser module
    .with_type("Interactive")  # With type Interactive
    .with_style({"tiles": "CartoDB dark_matter"}) # Dark theme style
    .show(columns=["avg_floors"])  # Show the avg_floors column
    .render(enriched_layer.get_layer()) # Render the enriched layer
)
fig_interactive

## Be Able To Preview Your Visualiser

Want to check your visualiser’s setup? `preview()` shows you the type and style—perfect for shared projects!

In [ ]:
# Preview visualiser
print(mapper.visual.preview())

# More visualiser primitives ?

Wants more? Come shout that out on https://github.com/VIDA-NYU/UrbanMapper/issues/9

## Wrapping Up

Blimey, you’re ace! 🌟 You’ve made static and interactive maps like a pro. Tweak the styles or columns next—you’re unstoppable! 🚀